# Техническое задание 3

**Дообучение автокорректора JamSpell на синтетически сгенерированных OCR-ошибках.**

Дообучить любой автокорректор на литературных текстах для улучшения качества коррекции ошибок при распознавании текста. В дообучении использовать тексты от 100 тыс. предложений. В ходе работы ТЗ дополнено: увеличить данные для обучения до 200 тыс. предложений.


## План работы

1. **Анализ основных ошибок в распознавании текста после работы модели Tesseract.**
- Загрузить тексты
- Вывести ошибки, после распознавания моделью текста. Проанализировать: какие ошибки пропускает модель.

2. **Подготовка текста для дообучения.**
* Выбор автокорректора для дообучения.
* Загрузка текста для генерации ошибок размером более 100 тыс.предложений.
* Сгенерировать OCR-ошибки
* Создание датасета

3. **Дообучение автокорректора JamSpell**
* Дообучение автокорректора JamSpell.
* Тестирование дообученной модели на тестовых данных
* Анализ метрик WER и CER после дообучения.

4. **Анализ и выводы о качестве работы дообученного автокорректора.**

## **Загрузка библиотек**

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

In [ ]:
pip install pytesseract pillow jiwer # установка библиотеки для вычисления метрик

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.6 MB/s eta 0:00:00


In [ ]:
import datasets
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
import json
import pandas as pd
from typing import List

In [ ]:
import random
import difflib
import re
import os
import string
import jiwer # библиотека для вычисления метрик

In [ ]:
# Установка seed для воспроизводимости
random.seed(42)

In [ ]:
pip install striprtf

In [ ]:
from striprtf.striprtf import rtf_to_text # Для открытия файлов rtf

#### **Библиотеки для JamSpell**

In [ ]:
!!sudo apt-get install swig3.0 # для JamSpell

['',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 37%',
 '',
 'Reading package lists... 37%',
 '',
 'Reading package lists... 37%',
 '',
 'Reading package lists... 37%',
 '',
 'Reading package lists... 37%',
 '',
 'Reading package lists... 44%',
 '',
 'Reading package lists... 44%',
 '',
 'Reading package lists... 54%',
 '',
 'Reading package lists... 54%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 65%',
 '',
 'Reading package lists... 65%',
 '',
 'Reading pack

In [ ]:
!sudo pip install jamspell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jamspell: filename=jamspell-0.0.12-cp311-cp311-linux_x86_64.whl size=1785475 sha256=3556fa9626f55bf08f2b9ec77d5bde56d4ee18486df745480bd3b42582e9e2b7
  Stored in directory: /root/.cache/pip/wheels/2c/f7/b1/1314c3a8f44a8e2bbe9f90b2ae05c404068f797b7681e1163b
Successfully built jamspell


In [ ]:
!wget -O ru.bin https://github.com/bakwc/JamSpell-models/raw/master/ru.tar.gz
!tar -xzf ru.bin || echo "Ошибка: файл ru.bin не распакован"

--2025-04-07 09:37:31--  https://github.com/bakwc/JamSpell-models/raw/master/ru.tar.gz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bakwc/JamSpell-models/master/ru.tar.gz [following]
--2025-04-07 09:37:31--  https://raw.githubusercontent.com/bakwc/JamSpell-models/master/ru.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39663667 (38M) [application/octet-stream]
Saving to: ‘ru.bin’

ru.bin              100%[===================>]  37.83M   207MB/s    in 0.2s    

2025-04-07 09:37:32 (207 MB/s) - ‘ru.bin’ saved [39663667/39663667]



In [ ]:
import jamspell

# **1. Анализ ошибок после распознавания текста**

Необходимо проанализировать основные ошибки в распознавании текста после работы модели Tesseract. Для этого загрузим распознанные моделью в предыдущих работах  тексты, эталонные тексты и используем метрики WER и CER, а так же выведем список ошибок, которые пропустила модель.

## Загрузка текстов

In [ ]:
from google.colab import drive # подключение гугл диска с файлами
drive.mount('/content/drive')

Mounted at /content/drive


**Литературный текст**

In [ ]:
rec_lit_text = '/content/drive/My Drive/Files_for_project/rec_lit_text.txt'
rec_lit_text = open(rec_lit_text, 'r', encoding='utf-8').read()

In [ ]:
etalon_lit_text = '/content/drive/My Drive/Files_for_project/etalon_lit_text.txt'
etalon_lit_text = open(etalon_lit_text, 'r', encoding='utf-8').read()

In [ ]:
print(rec_lit_text[0:500])
print(etalon_lit_text[0:500])

по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический 
﻿по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаически

**Учебный текст**

In [ ]:
rec_educ_text = '/content/drive/My Drive/Files_for_project/rec_educ_text.txt'
rec_educ_text = open(rec_educ_text, 'r', encoding='utf-8').read()

In [ ]:
etalon_educ_text = '/content/drive/My Drive/Files_for_project/etalon_educ_text.txt'
etalon_educ_text = open(etalon_educ_text, 'r', encoding='utf-8').read()

In [ ]:
print(rec_educ_text[0:500])
print(etalon_educ_text[0:500])

Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной ме
﻿Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; б) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной 

## Анализ ошибок при распознавании текста моделью

In [ ]:
def clean_text(text):
    """Удаляет знаки пунктуации из текста, кроме точек в сокращениях."""
    return re.sub(r"[.,!?;:\"'()\—]", "", text)

In [ ]:
def find_errors(ocr_text, reference_text):
    """Находит ошибки OCR: замененные, пропущенные и лишние слова."""

    # Очистка от запятых, тире и других знаков
    ocr_text = clean_text(ocr_text)
    reference_text = clean_text(reference_text)

    ocr_words = ocr_text.split()
    reference_words = reference_text.split()

    matcher = difflib.SequenceMatcher(None, reference_words, ocr_words)

    errors = []

    for opcode, i1, i2, j1, j2 in matcher.get_opcodes():
        if opcode == "replace":  # Слова заменены
            ref_part = " ".join(reference_words[i1:i2])
            ocr_part = " ".join(ocr_words[j1:j2])
            errors.append((ocr_part, ref_part, "замена"))

        elif opcode == "delete":  # OCR пропустил слово
            ref_part = " ".join(reference_words[i1:i2])
            errors.append(("[пропущено]", ref_part, "пропуск"))

        elif opcode == "insert":  # OCR добавил лишнее слово
            ocr_part = " ".join(ocr_words[j1:j2])
            errors.append((ocr_part, "[не должно быть]", "лишнее"))

    return errors

In [ ]:
errors_lit_text = find_errors(rec_lit_text, etalon_lit_text)
for ocr_word, ref_word, error_type in errors_lit_text:
    print(f"Ошибка ({error_type}): '{ocr_word}' → Должно быть: '{ref_word}'")

Ошибка (замена): 'по-видимому' → Должно быть: '﻿по-видимому'
Ошибка (замена): 'заинвентаризовано' → Должно быть: 'заинвентаризировано'
Ошибка (замена): 'паиболее' → Должно быть: 'наиболее'
Ошибка (замена): 'нище' → Должно быть: 'нише'
Ошибка (замена): 'ктото' → Должно быть: 'кто-то'
Ошибка (замена): 'спаружи' → Должно быть: 'снаружи'
Ошибка (замена): 'десять пятнадцать' → Должно быть: 'десять-пятнадцать'
Ошибка (замена): 'оптнмизм' → Должно быть: 'оптимизм'
Ошибка (замена): 'экстракфты' → Должно быть: 'экстракты'
Ошибка (замена): 'единой' → Должно быть: 'одной'
Ошибка (замена): 'злободроби тель' → Должно быть: 'злободробитель'
Ошибка (замена): 'собою' → Должно быть: 'собой'
Ошибка (замена): 'вареную' → Должно быть: 'вареную…'
Ошибка (замена): 'вниварии' → Должно быть: 'виварии'
Ошибка (замена): '‘сипло' → Должно быть: 'сипло'
Ошибка (замена): 'СТОЙЛ' → Должно быть: 'стойл'
Ошибка (замена): 'ли' → Должно быть: 'ли…'
Ошибка (замена): 'в глубь' → Должно быть: 'вглубь'
Ошибка (замена): 'го

In [ ]:
errors_educ_text = find_errors(rec_educ_text, etalon_educ_text)
for ocr_word, ref_word, error_type in errors_educ_text:
    print(f"Ошибка ({error_type}): '{ocr_word}' → Должно быть: '{ref_word}'")

Ошибка (замена): 'Субъектом' → Должно быть: '﻿Субъектом'
Ошибка (замена): '6' → Должно быть: 'б'
Ошибка (замена): '«исследователь»' → Должно быть: 'исследователь'
Ошибка (замена): 'социальноценностный' → Должно быть: 'социально-ценностный'
Ошибка (лишнее): '--' → Должно быть: '[не должно быть]'
Ошибка (замена): '«Закон' → Должно быть: 'Закон'
Ошибка (замена): 'добродетели»' → Должно быть: 'добродетели1'
Ошибка (замена): 'В Шершеневич' → Должно быть: 'Шершейевич'
Ошибка (замена): 'оппози' → Должно быть: 'оппози-'


**РЕЗУЛЬТАТ:**

Из анализа видно, что больше всего ошибок составляют OCR-ошибки. Например:
- 'паиболее' → Должно быть: 'наиболее'
- 'нище' → Должно быть: 'нише'
- 'ктото' → Должно быть: 'кто-то'
- 'спаружи' → Должно быть: 'снаружи'

Таким образом, вижу необходимость дообучить модель **на синтетически сгенерированных OCR-ошибках**.


## Функции для расчета метрик WER и CER

In [ ]:
def preprocess_text(text):
    """
    Функция для обработки текста для CER
    Приводит текст к нижнему регистру и удаляет знаки препинания и пробелы.
    """
    return ''.join(c.lower() for c in text if c not in string.punctuation and not c.isspace())

In [ ]:
def metrics_wer_cer(etalon_text, recognized_text):
    """
    Вычисление метрик CER и WER
    """
    # Обработка текстов для CER
    etalon_text_prep = preprocess_text(etalon_text)
    recognized_text_prep = preprocess_text(recognized_text)

    # Вычисление CER и WER
    wer = jiwer.wer(etalon_text, recognized_text)
    cer = jiwer.cer(etalon_text_prep, recognized_text_prep)

    return round(wer, 3), round(cer, 3)

# **2. Подготовка текста для дообучения**

## Загрузка текстов для генерации ошибок

**Загрузка текстов для обучения на 100 тысяч предложениях.**

**Загрузка файла Экономика**

In [ ]:
# путь к файлу
text_path = '/content/drive/My Drive/Files_for_project/Books/economica_all.rtf'

In [ ]:
# Загрузка RTF-файла
with open(text_path, "r", encoding="utf-8") as file:
    rtf_text = file.read()

# Преобразование RTF в обычный текст
text = rtf_to_text(rtf_text)
print(text[0:4000])

Институты.
Существует мнение, что институты — это не про Россию. Ведь институты — это некие правила.
Если речь идет о формальных правилах, о законах, то в России закон что дышло, и жизнь тут идет не по законам. Может быть, она идет по каким-то неписаным правилам? Но очевидное соблюдение таких правил характерно там, где есть сообщества людей со своими обычаями или нормами поведения, например в деревне. Но вряд ли кто-то всерьез полагает, что мы соблюдаем правила русской деревни: три раза что-то предлагаем, два раза отказываемся и тому подобное. Похоже, что и эти правила в России не работают. Иногда говорят, что мы живем по понятиям (не случайно же с 1950-х страна слушает уголовную лирику). Конечно, «понятия» — тоже институт, неформальные правила, которые поддерживаются преступными сообществами. И такие сообщества в России прошли очень серьезную закалку, ведь они возникли и выживали в период тоталитаризма, в 1930–40-е годы (точно так же, как в Италии мафия необычайно укрепилась, кристалл

In [ ]:
def remove_hyphens(text):
    '''Убрает переносы'''
    # Убираем переносы слов, заменяя '-\n' на пустую строку
    text = re.sub(r'-\n', '', text)
    # Убираем переносы строк, если они не следуют за переносом слова
    text = re.sub(r'\n', ' ', text)
    return text

In [ ]:
text_econom = remove_hyphens(text) # убираем переносы
print(text_econom[0:5000]) # печать части текста, для проверки

Институты. Существует мнение, что институты — это не про Россию. Ведь институты — это некие правила. Если речь идет о формальных правилах, о законах, то в России закон что дышло, и жизнь тут идет не по законам. Может быть, она идет по каким-то неписаным правилам? Но очевидное соблюдение таких правил характерно там, где есть сообщества людей со своими обычаями или нормами поведения, например в деревне. Но вряд ли кто-то всерьез полагает, что мы соблюдаем правила русской деревни: три раза что-то предлагаем, два раза отказываемся и тому подобное. Похоже, что и эти правила в России не работают. Иногда говорят, что мы живем по понятиям (не случайно же с 1950-х страна слушает уголовную лирику). Конечно, «понятия» — тоже институт, неформальные правила, которые поддерживаются преступными сообществами. И такие сообщества в России прошли очень серьезную закалку, ведь они возникли и выживали в период тоталитаризма, в 1930–40-е годы (точно так же, как в Италии мафия необычайно укрепилась, кристалл

In [ ]:
def qty_sentences(text):
    '''Подсчет количества предложений'''
    # Разделение текста на предложения по знакам конца предложения (., !, ?)
    sentences = re.split(r'[.!?]\s+', text)
    # Подсчёт количества предложений
    num_sentences = len(sentences)
    print(f"Количество предложений в тексте: {num_sentences}")
    return sentences

In [ ]:
def show_sentences(text, num_sentences):
    '''Вывод предложений на экран'''
    sentences = qty_sentences(text)
    for sentence in sentences[:num_sentences]:
        print(sentence.strip())

In [ ]:
show_sentences(text_econom, 3)

Количество предложений в тексте: 980
Институты
Существует мнение, что институты — это не про Россию
Ведь институты — это некие правила


In [ ]:
def load_texts_from_folder(folder_path, encoding='windows-1251'):
    """
    Загружает все текстовые файлы из указанной папки

    :param folder_path: путь к папке с файлами
    :param encoding: кодировка файлов (по умолчанию windows-1251)
    :return: словарь {имя_файла: содержимое}
    """
    texts = {}
    # Перебираем все файлы в папке
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        # Загружаем только текстовые файлы
        if os.path.isfile(file_path) and filename.endswith('.txt'):
            with open(file_path, 'r', encoding=encoding) as file:
                lines = [line.strip() for line in file.readlines()]
                text = '\n'.join(lines)
                # Ваша функция для обработки переносов
                texts[filename] = remove_hyphens(text)
    return texts

In [ ]:
folder_path = '/content/drive/My Drive/Files_for_project/Books_100/'
all_texts = load_texts_from_folder(folder_path)
# Печатаем имена загруженных файлов и первые 3 предложения
for name, text in all_texts.items():
    print(f"\nФайл: {name}")
    print(f"Начало текста: {text[:100]}...")


Файл: Frayi_Labirintyi_Eho.txt
Начало текста: Меня действительно зовут Макс. Сколько себя помню, всегда предпочитал сокращенный вариант собственно...

Файл: Sanderson_Legion.txt
Начало текста: Психология как суперсила – тема, которая в моих произведениях повторяется снова и снова. Я всегда сч...

Файл: Akunin_Nefritovyie_Chetki.txt
Начало текста: На похоронах человека, который собирался стать буддой, публики было до неприличного мало. Из компатр...

Файл: Simmons_Giperion.txt
Начало текста: Консул Гегемонии сидел на балконе своего эбеново-черного космического корабля и на хорошо сохранивше...

Файл: Aristotel_Ritorika.txt
Начало текста: Аристотель – величайший ученый своего времени, философ и практик, работы которого стали основой разв...

Файл: Lukyanenko_Dozoryi.txt
Начало текста: Эскалатор полз медленно, натужно. Старая станция, ничего не поделаешь. Зато ветер гулял в бетонной т...


In [ ]:
# Объединить тексты без разделителей и удалить '\xa0'
combined_text_100 = "".join(all_texts.values())
combined_text_100 = combined_text_100.replace('\xa0', '')
print(combined_text_100[:100])

Меня действительно зовут Макс. Сколько себя помню, всегда предпочитал сокращенный вариант собственно


In [ ]:
# соединение с файлом по экономике
total_text_100 = " ".join([combined_text_100, text_econom])

In [ ]:
show_sentences(total_text_100, 3)

Количество предложений в тексте: 126877
Меня действительно зовут Макс
Сколько себя помню, всегда предпочитал сокращенный вариант собственного имени
Я родом откуда-то из этих мест


**Загрузка текстов для обучения на 200 тысяч предложениях.**

*Загрузка художественных книг:*

- Макс Фрай. Лабиринты Ехо
- Акунин. Нефритовые четки
- Хейли. Отель
- Сандерсон. Легион
- Аристотель. Риторика
- Дефо. Робинзон Крузо
- Сервантес. Дон Кихот
- Верн. Таинственный остров
- Симмонс. Гипперион
- Лукьяненко. Дозоры
- Гейман. Благие знамения
- Адамс. Автостопом по галактике, 2 части


In [ ]:
folder_path = '/content/drive/My Drive/Files_for_project/Books/'
all_texts = load_texts_from_folder(folder_path)
# Печатаем имена загруженных файлов и первые 3 предложения
for name, text in all_texts.items():
    print(f"\nФайл: {name}")
    print(f"Начало текста: {text[:100]}...")


Файл: Frayi_Labirintyi_Eho.txt
Начало текста: Меня действительно зовут Макс. Сколько себя помню, всегда предпочитал сокращенный вариант собственно...

Файл: Heyili_Otel.txt
Начало текста: 1  «Будь на то моя воля, – думал Питер Макдермотт, – я давным-давно выгнал бы этого начальника охран...

Файл: Akunin_Nefritovyie_Chetki.txt
Начало текста: На похоронах человека, который собирался стать буддой, публики было до неприличного мало. Из компатр...

Файл: Sanderson_Legion.txt
Начало текста: Психология как суперсила – тема, которая в моих произведениях повторяется снова и снова. Я всегда сч...

Файл: Aristotel_Ritorika.txt
Начало текста: Аристотель – величайший ученый своего времени, философ и практик, работы которого стали основой разв...

Файл: Defo_Robinzon_Kruzo.txt
Начало текста: Если существует на свете история приключений частного лица, заслуживающая стать всеобщим достоянием ...

Файл: Vern_Tainstvennyiyi_Ostrov.txt
Начало текста: – Мы поднимаемся?  – Нет, наоборот, мы опускаемся!  

In [ ]:
# Объединить тексты без разделителей и удалить '\xa0'
combined_text = "".join(all_texts.values())
combined_text = combined_text.replace('\xa0', '')
print(combined_text[:100])

Меня действительно зовут Макс. Сколько себя помню, всегда предпочитал сокращенный вариант собственно


In [ ]:
show_sentences(combined_text, 3)

Количество предложений в тексте: 243054
Меня действительно зовут Макс
Сколько себя помню, всегда предпочитал сокращенный вариант собственного имени
Я родом откуда-то из этих мест


In [ ]:
total_text = " ".join([combined_text, text_econom])  # соединение с файлом по экономике

In [ ]:
print(total_text[:1000])

Меня действительно зовут Макс. Сколько себя помню, всегда предпочитал сокращенный вариант собственного имени.  Я родом откуда-то из этих мест. Возможно даже, ваш бывший сосед. Я прожил здесь около тридцати лет, пока не попал в Ехо.  Вы не найдете город Ехо ни на одной карте. Потому как Ехо находится не на этой планете. Точнее сказать, не в этом Мире. Второе определение мне нравится больше, поскольку выражение «не на этой планете» неизбежно ассоциируется с полетами на космических кораблях, чего со мной, по счастию, никогда не происходило. Подробный отчет о моем путешествии в Ехо вы найдете в одной из историй, за которые приметесь после того, как покончите с предисловием. Она называется «Чужак».  Итак, город Ехо – это столица Соединенного Королевства Угуланда, Гугланда, Ландаланда и Уриуланда, а также графств Шимара и Вук, земель Благостного Ордена Семилистника, вольного города Гажин и острова Муримах.  Далее. Законы природы этого Мира не только допускают, но даже провоцируют развитие та

In [ ]:
show_sentences(total_text, 8)

Количество предложений в тексте: 244033
Меня действительно зовут Макс
Сколько себя помню, всегда предпочитал сокращенный вариант собственного имени
Я родом откуда-то из этих мест
Возможно даже, ваш бывший сосед
Я прожил здесь около тридцати лет, пока не попал в Ехо
Вы не найдете город Ехо ни на одной карте
Потому как Ехо находится не на этой планете
Точнее сказать, не в этом Мире


**РЕЗУЛЬТАТ:**

Загружены тексты для генерации ошибок, объединены в один файл, и разделен на отдельные предложения.  

Сделано 2 варианта по объему текста: на 126877 и 244041 предложение.

## Генерация OCR-ошибок

Для генерации OCR-ошибок использованы самые распространенные варианты ошибок и объединены в словарь.

In [ ]:
# Словарь для замены символов
# (ключ: правильный символ, значение: список возможных замен)
OCR_ERRORS = {
    "о": ["0", "а"],
    "а": ["@", "о"],
    "е": ["ё"],
    "ё": ["е"],
    "ш": ["щ"],
    "м": ["п"],
    "г": ["т"],
    "т": ["г"],
    "с": ["е", "о"],
    "у": ["ц", "ч"],
    "к": ["ж", "х"],
    "х": ["к", "ж"],
    "ъ": ["ь"],
    "ь": ["ъ"],
    "й": ["и"],
    "и": ["н", "й"],
    "н": ["и", "п"],
    "в": ["8", "ь", "ъ"],
    "а": ["@"],
    "з": ["3"],
    "ч": ["4"],
    "б": ["6"],

}

In [ ]:
def generate_ocr_errors(text, error_prob=0.1, max_errors_per_word=1):
    """
    Генерирует OCR-ошибки в тексте с ограничением на количество ошибок в слове.
    :param text: Исходный текст.
    :param error_prob: Вероятность ошибки для каждого символа.
    :param max_errors_per_word: Максимальное количество ошибок в одном слове.
    :return: Текст с ошибками.
    """
    words = text.split(' ')
    result = []

    for word in words:
        # Счетчик ошибок в текущем слове
        error_count = 0
        modified_word = []

        for char in word:
            # Проверяем условия для добавления ошибки
            if (char.lower() in OCR_ERRORS and
                random.random() < error_prob and
                error_count < max_errors_per_word):
                # Заменяем символ на случайный из списка возможных замен
                replacement = random.choice(OCR_ERRORS[char.lower()])
                # Сохраняем регистр
                if char.isupper():
                    replacement = replacement.upper()
                modified_word.append(replacement)
                error_count += 1
            else:
                modified_word.append(char)

        result.append(''.join(modified_word))

    return ' '.join(result)

**Датасет на 100 тысяч предложений.**

In [ ]:
# Генерация 10% вероятность ошибки
noisy_text_100 = generate_ocr_errors(total_text_100, error_prob=0.1)
print(f"Исходный текст:   {total_text_100[0:3000]}")
print(f"Текст с ошибками: {noisy_text_100[0:3000]}")

Исходный текст:   Меня действительно зовут Макс. Сколько себя помню, всегда предпочитал сокращенный вариант собственного имени.  Я родом откуда-то из этих мест. Возможно даже, ваш бывший сосед. Я прожил здесь около тридцати лет, пока не попал в Ехо.  Вы не найдете город Ехо ни на одной карте. Потому как Ехо находится не на этой планете. Точнее сказать, не в этом Мире. Второе определение мне нравится больше, поскольку выражение «не на этой планете» неизбежно ассоциируется с полетами на космических кораблях, чего со мной, по счастию, никогда не происходило. Подробный отчет о моем путешествии в Ехо вы найдете в одной из историй, за которые приметесь после того, как покончите с предисловием. Она называется «Чужак».  Итак, город Ехо – это столица Соединенного Королевства Угуланда, Гугланда, Ландаланда и Уриуланда, а также графств Шимара и Вук, земель Благостного Ордена Семилистника, вольного города Гажин и острова Муримах.  Далее. Законы природы этого Мира не только допускают, но даже прово

In [ ]:
# Разбиваем текст на предложения
def divide_sentences(text):
    sentences = re.split(r'[.!?]\s+', text)
    sentences = list(map(lambda x: x.replace("\xa0", " "), sentences))
    return sentences

In [ ]:
# Создаем датасет
dataset_total_100 = Dataset.from_dict({
    "source": divide_sentences(noisy_text_100),  # Текст с ошибками
    "correction": divide_sentences(total_text_100)  # Исправленный текст
})
print(dataset_total_100[0:15])

{'source': ['Меия действительно зовут Пакс', 'Околько себя помню, всегда предпочитал сокращенный 8ариант собственного имени', 'Я родом огкуда-то из этих мест', '8озможно д@же, ваш 6ывший сосед', 'Я прожил 3десь около тридцати лёт, пока не попал в Ехо', 'Вы не найдете город Ёхо ни на однои карте', 'Потому как Еко нажодится не на этой планете', 'Точнее сказать, не в этом Мире', 'Второе определение мне нр@вится больше, посколъку выражение «ие на этай планете» неизбёжно ассоциируегся е полет@ми па жосмических кора6лях, чего со мной, п0 счасгию, никогда не происходило', 'Подробный отчет 0 моем пчтешествии в Ехо 8ы найдете в одной из нсторий, за к0торые приметесъ пасле гого, как покончите е предйсловием', '0на называется «Чужак»', 'Итак, город Ёхо – это столица Соединенного Королеветва Утуланда, Гугланд@, Ланд@ланда и Уриуланда, а также графств Шимар@ и Вук, земёль Благостного 0рдена Сепилистника, вольного города Гажин и осгрова Муримах', 'Далеё', 'З@коны природы этого Мира не только допуока

In [ ]:
# Разделение с перемешиванием и фиксированным random_state
dataset_total_100 = dataset_total_100.shuffle(seed=42)  # Перемешиваем данные
dataset_total_100 = dataset_total_100.train_test_split(
    test_size=0.2,
    seed=42,  # Фиксируем random_state
    shuffle=True  # Включено по умолчанию, но можно явно указать
)

# Проверка структуры
print(dataset_total_100)

DatasetDict({
    train: Dataset({
        features: ['source', 'correction'],
        num_rows: 101501
    })
    test: Dataset({
        features: ['source', 'correction'],
        num_rows: 25376
    })
})


In [ ]:
# Путь для сохранения в Google Drive
save_path_100 = '/content/drive/My Drive/Files_for_project/ocr_dataset_100'

# Сохранение датасета
dataset_total_100.save_to_disk(save_path_100)

print(f"Датасет сохранён в: {save_path_100}")

Saving the dataset (0/1 shards):   0%|          | 0/101501 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25376 [00:00<?, ? examples/s]

Датасет сохранён в: /content/drive/My Drive/Files_for_project/ocr_dataset_100


**РЕЗУЛЬТАТ:**

Сгенерированы самые распространенные OCR-ошибки для русского языка с вероятностью ошибки 10% на основе оригинального текста размером более 100 тысяч предложений.

**Датасет на 200 тысяч предложений.**

In [ ]:
# Генерация 10% вероятность ошибки
noisy_text = generate_ocr_errors(total_text, error_prob=0.1)
print(f"Исходный текст:   {total_text[0:3000]}")
print(f"Текст с ошибками: {noisy_text[0:3000]}")

Исходный текст:   Меня действительно зовут Макс. Сколько себя помню, всегда предпочитал сокращенный вариант собственного имени.  Я родом откуда-то из этих мест. Возможно даже, ваш бывший сосед. Я прожил здесь около тридцати лет, пока не попал в Ехо.  Вы не найдете город Ехо ни на одной карте. Потому как Ехо находится не на этой планете. Точнее сказать, не в этом Мире. Второе определение мне нравится больше, поскольку выражение «не на этой планете» неизбежно ассоциируется с полетами на космических кораблях, чего со мной, по счастию, никогда не происходило. Подробный отчет о моем путешествии в Ехо вы найдете в одной из историй, за которые приметесь после того, как покончите с предисловием. Она называется «Чужак».  Итак, город Ехо – это столица Соединенного Королевства Угуланда, Гугланда, Ландаланда и Уриуланда, а также графств Шимара и Вук, земель Благостного Ордена Семилистника, вольного города Гажин и острова Муримах.  Далее. Законы природы этого Мира не только допускают, но даже прово

In [ ]:
# Создаем датасет
dataset_total = Dataset.from_dict({
    "source": divide_sentences(noisy_text),  # Текст с ошибками
    "correction": divide_sentences(total_text)  # Исправленный текст
})
print(dataset_total[0:15])

{'source': ['Мёня действнтельно зовуг Макс', 'Сжолько себя памню, всетда предпочитал сокращеиный вариант с0бственного имени', 'Я родом откуда-то из этих мест', 'Возможно д@же, ваш бывший сосед', 'Я прожил здесь окола тридцати лет, пока не попал в Ехо', 'Вы нё найдёте г0род Ехо ни на одпой к@рте', 'Потому как Ёхо находится нё па этой планете', 'Точнеё сказ@ть, не в этоп Мире', 'Второе определепие пне нр@вится больще, поскольку выражение «нё н@ этой планеге» нейзбежно ассоциируется с полетамй на космичёских кор@блях, 4его со пной, по счастию, ник0гда не происходило', 'Подробный отчег о моем путешествин в Ех0 вы найдете в одной из историй, за которые приметесь после того, как покон4ите о предиоловием', 'Она называется «Чужак»', 'Итак, город Ехо – эго сголица Соединенпого Жоролевства Угуланда, Гугл@нда, Ландаланда и Уриуланд@, а гакже графств Шимара и Вук, земель Благостного Ордена Семилистнйка, вольного города Гажин и острова Мурим@х', 'Далее', 'Законы природы этаго Мира не т0лько допуска

In [ ]:
dataset_total

Dataset({
    features: ['source', 'correction'],
    num_rows: 244033
})

Создала датасет с текстом с ошибками и эталонным текстом. Разделим на train и test

In [ ]:
# Разделение с перемешиванием и фиксированным random_state
dataset_total = dataset_total.shuffle(seed=42)  # Перемешиваем данные
dataset_total = dataset_total.train_test_split(
    test_size=0.2,
    seed=42,  # Фиксируем random_state
    shuffle=True  # Включено по умолчанию, но можно явно указать
)

# Проверка структуры
print(dataset_total)

DatasetDict({
    train: Dataset({
        features: ['source', 'correction'],
        num_rows: 195226
    })
    test: Dataset({
        features: ['source', 'correction'],
        num_rows: 48807
    })
})


In [ ]:
# Путь для сохранения в Google Drive
save_path = '/content/drive/My Drive/Files_for_project/ocr_dataset'

# Сохранение датасета
dataset_total.save_to_disk(save_path)

print(f"Датасет сохранён в: {save_path}")

Saving the dataset (0/1 shards):   0%|          | 0/195226 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/48807 [00:00<?, ? examples/s]

Датасет сохранён в: /content/drive/My Drive/Files_for_project/ocr_dataset


**РЕЗУЛЬТАТ:**

Сгенерированы самые распространенные OCR-ошибки для русского языка с вероятностью ошибки 10% на основе оригинального текста размером более 200 тысяч предложений.

# **3. Дообучение моделей**

## Дообучение модели JamSpell

**Дообучение на 100 тыс. предложениях.**

Извлечь данные из DatasetDict. Преобразовать данные в формат, понятный JamSpell (список кортежей (текст с ошибками -> правильный текст)).



In [ ]:
# Инициализация JamSpell и загрузка предобученной модели для 100 тыс.предлож.
corrector_100 = jamspell.TSpellCorrector()
corrector_100.LoadLangModel("ru_small.bin")
print("Предобученная модель 'ru_small.bin' успешно загружена!")

Предобученная модель 'ru_small.bin' успешно загружена!


In [ ]:
def export_to_text(dataset, output_file):
    """
    Экспортирует Dataset в текстовый файл. Каждая строка - исправленный текст.
    """
    with open(output_file, "w", encoding="utf-8") as f:
        #for sample in dataset_total['train']:
        for sample in dataset:
            ocr_text = sample['source']        # Текст с OCR-ошибками
            correct_text = sample['correction']  # Эталонный текст
            # Пишем пару OCR -> исправление
            f.write(f"{ocr_text} -> {correct_text}\n")

In [ ]:
# Извлекаем и сохраняем train/test в текстовые файлы на 100 тыс. предложений
train_file_100 = "train_100.txt"
test_file_100 = "test_100.txt"
export_to_text(dataset_total_100["train"], train_file_100)
export_to_text(dataset_total_100["test"], test_file_100)
print(f"Данные сохранены:\n - Train: {train_file_100}\n - Test: {test_file_100}")

Данные сохранены:
 - Train: train_100.txt
 - Test: test_100.txt


In [ ]:
# Чтение первых 5 строк из файла и вывод на экран
with open(train_file_100, "r", encoding="utf-8") as file:
    for i in range(5):  # Чтение и вывод первых 5 строк
        # Считываем строку и убираем лишние пробелы
        line = file.readline().strip()
        print(line)

–Это ее гробница, отец -> –Это ее гробница, отец
Славное местечко -> Славное местечко
Ои тут же явился во главе толпы скульпторов -> Он тут же явился во главе толпы скульпторов
Мне как-го довелось увидеть одпо нз 4удовищных пол0тен знаменигого придьорного живописца Гуриг@ VII – портрег боеваго тенерала Бубуты на фоне какой-то ъеликой битвы -> Мне как-то довелось увидеть одно из чудовищных полотен знаменитого придворного живописца Гурига VII – портрет боевого генерала Бубуты на фоне какой-то великой битвы
–Пет -> –Нет


In [ ]:
# Дообучение JamSpell на OCR-ошибках (с использованием тренировочного файла)
alphabet_file = "alphabet.txt"
with open(alphabet_file, "w", encoding="utf-8") as f:
    alphabet = "абвгдежзийклмнопрстуфхцчшщъыьэюяё"
    f.write("\n".join(alphabet))

Дообучить модель JamSpell на этих данных. Сохранить обновленную модель.

In [ ]:
# Дообучение модели (на OCR-ошибках)
model_file_100 = "updated_ocr_model_100.bin"
corrector_100.TrainLangModel(train_file_100, alphabet_file, model_file_100)
print(f"Дообученная модель сохранена в '{model_file_100}'!")

Дообученная модель сохранена в 'updated_ocr_model_100.bin'!


#### **Тестирование дообученной модели на тестовых данных на 100 тыс. предложениях**

In [ ]:
# Тестирование дообученной модели
new_corrector_100 = jamspell.TSpellCorrector()
new_corrector_100.LoadLangModel(model_file_100)  # Загрузка дообученной модели

True

In [ ]:
# Открытие файла тестовых данных
# Файл с тестовыми текстами (текст с ошибками -> эталонный текст)
test_file_path = "test_100.txt"
results_file_path = "test_results_100.txt"  # Файл для записи результатов

# Запись результатов тестирования
with open(test_file_path,
          "r",
          encoding="utf-8") as test_file,
          open(results_file_path,
              "w",
              encoding="utf-8") as results_file:
    for line in test_file:
        line = line.strip()  # Убираем лишние пробелы и переводы строк

        if " -> " not in line:
            print(f"Ошибка в строке: {line}")  # Выводим проблемную строку
            continue  # Пропускаем строки, которые не соответствуют формату
        # Разделяем OCR текст и эталонный текст
        ocr_text, correct_text = line.split(" -> ", 1)
        # Исправление текста с помощью дообученной модели
        corrected_text = new_corrector_100.FixFragment(ocr_text)

        # Запись результатов в файл
        results_file.write(f"OCR текст:        {ocr_text}\n")
        results_file.write(f"Эталонный текст:  {correct_text}\n")
        results_file.write(f"Откоррект. текст: {corrected_text}\n")

        # Пустая строка между блоками для удобства чтения
        results_file.write("\n")
print(f"Результаты тестирования сохранены в '{results_file_path}'!")

Результаты тестирования сохранены в 'test_results_100.txt'!


Вывод результатов дообученной модели по сравнению с эталонным текстом и текстом с ошибками.

In [ ]:
# Чтение первых строк из файла и вывод на экран
file_res_path = "test_results_100.txt"  # Путь к файлу

with open(file_res_path, "r", encoding="utf-8") as file:
    for i in range(24):  # Чтение и вывод первых строк
        # Считываем строку и убираем лишние пробелы
        line = file.readline().strip()
        print(line)

OCR текст:        Ее черные гла3а сматрели на нас насгороженно и даже немного сердито
Эталонный текст:  Ее черные глаза смотрели на нас настороженно и даже немного сердито
Откоррект. текст: Ее черные гла3а сматрели на нас настороженно и даже немного сердито

OCR текст:        Я и сообразить-то ничего не уепел
Эталонный текст:  Я и сообразить-то ничего не успел
Откоррект. текст: Я и сообразить-то ничего не уепел

OCR текст:        О 0боротнями никогда не цгадаешь… А 4его ты так цдивился
Эталонный текст:  С оборотнями никогда не угадаешь… А чего ты так удивился
Откоррект. текст: О 0боротнями никогда не угадаешь… А 4его ты так цдивился

OCR текст:        –Лучше бы ты любйл, чем берёг
Эталонный текст:  –Лучше бы ты любил, чем берег
Откоррект. текст: –Лучше бы ты любил, чем берёг

OCR текст:        –Как угодпо.– Завулон пож@л плёчами.– Повторяю, пока мы прёдлагаем свои чслуги
Эталонный текст:  –Как угодно.– Завулон пожал плечами.– Повторяю, пока мы предлагаем свои услуги
Откоррект. текст: –

In [ ]:
# Проверим другой текст, должен исправить ошибки
print(new_corrector_100.FixFragment("Тексмт с ошбиками. Превет! Эжономисты ачень долго не котели абсуждать роль общества в экономике"))

Текст с ошбиками. Привет! Экономисты ачень долго не котели обсуждать роль общества в экономике


**Дообучение на 200 тыс. предложениях.**

In [ ]:
# Инициализация JamSpell и загрузка предобученной модели для 200 тыс.предложений
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel("ru_small.bin")
print("Предобученная модель 'ru_small.bin' успешно загружена!")

Предобученная модель 'ru_small.bin' успешно загружена!


In [ ]:
def export_to_text(dataset, output_file):
    """
    Экспортирует Dataset в текстовый файл. Каждая строка - исправленный текст.
    """
    with open(output_file, "w", encoding="utf-8") as f:
        #for sample in dataset_total['train']:
        for sample in dataset:
            ocr_text = sample['source']        # Текст с OCR-ошибками
            correct_text = sample['correction']  # Эталонный текст
            # Пишем пару OCR -> исправление
            f.write(f"{ocr_text} -> {correct_text}\n")

In [ ]:
# Извлекаем и сохраняем train/test в текстовые файлы на 200 тыс. предложений
train_file = "train.txt"
test_file = "test.txt"
export_to_text(dataset_total["train"], train_file)
export_to_text(dataset_total["test"], test_file)
print(f"Данные сохранены:\n - Train: {train_file}\n - Test: {test_file}")

Данные сохранены:
 - Train: train.txt
 - Test: test.txt


In [ ]:
# Чтение первых 5 строк из файла и вывод на экран
with open(train_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Чтение и вывод первых 5 строк
        # Считываем строку и убираем лишние пробелы
        line = file.readline().strip()
        print(line)

А вместе с людьми сп0койно и интересн0 -> А вместе с людьми спокойно и интересно
И тольжо Триллиан не подняла свой пистолет -> И только Триллиан не подняла свой пистолет
Внезапно грудь сгарика приподнялась -> Внезапно грудь старика приподнялась
Коридор увел вглубь д0ма, в квартиру Зуевых -> Коридор увел вглубь дома, в квартиру Зуевых
–Смотри -> –Смотри


In [ ]:
# Дообучение JamSpell на OCR-ошибках (с использованием тренировочного файла)
alphabet_file = "alphabet.txt"
with open(alphabet_file, "w", encoding="utf-8") as f:
    alphabet = "абвгдежзийклмнопрстуфхцчшщъыьэюяё"
    f.write("\n".join(alphabet))

Дообучить модель JamSpell на этих данных. Сохранить обновленную модель.

In [ ]:
# Дообучение модели (на OCR-ошибках)
model_file = "updated_ocr_model.bin"
corrector.TrainLangModel(train_file, alphabet_file, model_file)
print(f"Дообученная модель сохранена в '{model_file}'!")

Дообученная модель сохранена в 'updated_ocr_model.bin'!


#### **Тестирование дообученной модели на тестовых данных 200 тыс.предложений**

In [ ]:
# Тестирование дообученной модели
new_corrector = jamspell.TSpellCorrector()
new_corrector.LoadLangModel(model_file)  # Загрузка дообученной модели

True

In [ ]:
# Открытие файла тестовых данных
# Файл с тестовыми текстами (текст с ошибками -> эталонный текст)
test_file_path = "test.txt"
results_file_path = "test_results.txt"  # Файл для записи результатов

# Запись результатов тестирования
with open(test_file_path,
          "r",
          encoding="utf-8") as test_file,
          open(results_file_path,
               "w",
               encoding="utf-8") as results_file:
    for line in test_file:
        line = line.strip()  # Убираем лишние пробелы и переводы строк

        if " -> " not in line:
            print(f"Ошибка в строке: {line}")  # Выводим проблемную строку
            continue  # Пропускаем строки, которые не соответствуют формату
        # Разделяем OCR текст и эталонный текст
        ocr_text, correct_text = line.split(" -> ", 1)
        # Исправление текста с помощью дообученной модели
        corrected_text = new_corrector.FixFragment(ocr_text)

        # Запись результатов в файл
        results_file.write(f"OCR текст:        {ocr_text}\n")
        results_file.write(f"Эталонный текст:  {correct_text}\n")
        results_file.write(f"Откоррект. текст: {corrected_text}\n")
        # Пустая строка между блоками для удобства чтения
        results_file.write("\n")

print(f"Результаты тестирования сохранены в '{results_file_path}'!")

Ошибка в строке: ->
Результаты тестирования сохранены в 'test_results.txt'!


Вывод результатов дообученной модели по сравнению с эталонным текстом и текстом с ошибками.

In [ ]:
# Чтение первых строк из файла и вывод на экран
file_res_path = "test_results.txt"  # Путь к вашему файлу

with open(file_res_path, "r", encoding="utf-8") as file:
    for i in range(24):  # Чтение и вывод первых строк
        # Считываем строку и убираем лишние пробелы
        line = file.readline().strip()
        print(line)

OCR текст:        3начит, там, где растут эти дёревья, вечпые лнхорадки…  –Вовсе нет
Эталонный текст:  Значит, там, где растут эти деревья, вечные лихорадки…  –Вовсе нет
Откоррект. текст: 3начит, там, где растут эти дёревья, вечные лнхорадки…  –Вовсе нет

OCR текст:        –А когда это я ныл по пустякам
Эталонный текст:  –А когда это я ныл по пустякам
Откоррект. текст: –А когда это я был по пустякам

OCR текст:        К моему и квашему несчастью, я не в силах освободить вас из злого плена
Эталонный текст:  К моему и квашему несчастью, я не в силах освободить вас из злого плена
Откоррект. текст: К моему и вашему несчастью, я не в силах освободить вас из злого плена

OCR текст:        В домашней куртке, в мягких туфлях, то есгь сапого безмятёжного вида
Эталонный текст:  В домашней куртке, в мягких туфлях, то есть самого безмятежного вида
Откоррект. текст: В домашней куртке, в мягких туфлях, то есгь сапого безмятёжного вида

OCR текст:        Жээ, сказал бы кто-ни6удь Масахиро Сибаге во в

In [ ]:
# Проверим другой текст, должен исправить ошибки
print(new_corrector.FixFragment("Тексмт с ошбиками. Превет! Эжономисты ачень долго не котели абсуждать роль общества в экономике"))

Текст с ошбиками. Привет! Эжономисты ачень долго не котели абсуждать роль общества в экономике


**РЕЗУЛЬТАТ:**

Визуально видно, что дообучение прошло не очень эффективно и исправляет не все ошибки. Возможно, что объем текста недотаточен или тип модели не позволяет повысить эффективность.

# 4. **Анализ метрик после дообучения**

После дообучения проанализируем метрики и сравним: метрики текста с ошибками и метрики откорректированного текста после дообучения.

**Метрики для дообучения текстов на 100 тыс. предложениях.**

In [ ]:
# Инициализация пустых списков
ocr_texts_100 = []
correct_texts_100 = []
corrected_texts_100 = []

# Чтение файла результатов
with open("test_results_100.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()
    i = 0
    while i < len(lines):
        # Ищем строку с OCR текстом
        if lines[i].startswith("OCR текст:"):
            # Извлекаем OCR текст (удаляем префикс и лишние пробелы)
            ocr = lines[i].split(":", 1)[1].strip()
            ocr_texts_100.append(ocr)

            # Извлекаем эталонный текст (следующая строка)
            correct = lines[i+1].split(":", 1)[1].strip()
            correct_texts_100.append(correct)

            # Извлекаем исправленный текст (через строку)
            corrected = lines[i+2].split(":", 1)[1].strip()
            corrected_texts_100.append(corrected)

            # Пропускаем 4 строки (3 данные + 1 пустая)
            i += 4
        else:
            i += 1

# Проверка результатов
print(f"Извлечено {len(ocr_texts_100)} записей:")
print("OCR тексты:         ", ocr_texts_100[:3], "...")
print("Эталонные тексты:   ", correct_texts_100[:3], "...")
print("Исправленные тексты:", corrected_texts_100[:3], "...")

Извлечено 25376 записей:
OCR тексты:          ['Ее черные гла3а сматрели на нас насгороженно и даже немного сердито', 'Я и сообразить-то ничего не уепел', 'О 0боротнями никогда не цгадаешь… А 4его ты так цдивился'] ...
Эталонные тексты:    ['Ее черные глаза смотрели на нас настороженно и даже немного сердито', 'Я и сообразить-то ничего не успел', 'С оборотнями никогда не угадаешь… А чего ты так удивился'] ...
Исправленные тексты: ['Ее черные гла3а сматрели на нас настороженно и даже немного сердито', 'Я и сообразить-то ничего не уепел', 'О 0боротнями никогда не угадаешь… А 4его ты так цдивился'] ...


Метрики для текста сгенерированных OCR-ошибок.

In [ ]:
wer_ocr_100, cer_ocr_100 = metrics_wer_cer(correct_texts_100,
                                           ocr_texts_100)
print(f"WER: {wer_ocr_100}")
print(f"CER: {cer_ocr_100}")

WER: 0.326
CER: 0.051


Метрики для текста с автокоррекцией с дообученной моделью.

In [ ]:
wer_corr_100, cer_corr_100 = metrics_wer_cer(correct_texts_100,
                                             corrected_texts_100)
print(f"WER: {wer_corr_100}")
print(f"CER: {cer_corr_100}")

WER: 0.295
CER: 0.048


**Метрики для дообучения текстов на 200 тыс. предложениях.**

In [ ]:
# Инициализация пустых списков
ocr_texts = []
correct_texts = []
corrected_texts = []

# Чтение файла результатов
with open("test_results.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()
    i = 0
    while i < len(lines):
        # Ищем строку с OCR текстом
        if lines[i].startswith("OCR текст:"):
            # Извлекаем OCR текст (удаляем префикс и лишние пробелы)
            ocr = lines[i].split(":", 1)[1].strip()
            ocr_texts.append(ocr)

            # Извлекаем эталонный текст (следующая строка)
            correct = lines[i+1].split(":", 1)[1].strip()
            correct_texts.append(correct)

            # Извлекаем исправленный текст (через строку)
            corrected = lines[i+2].split(":", 1)[1].strip()
            corrected_texts.append(corrected)

            # Пропускаем 4 строки (3 данные + 1 пустая)
            i += 4
        else:
            i += 1

# Проверка результатов
print(f"Извлечено {len(ocr_texts)} записей:")
print("OCR тексты:         ", ocr_texts[:3], "...")
print("Эталонные тексты:   ", correct_texts[:3], "...")
print("Исправленные тексты:", corrected_texts[:3], "...")

Извлечено 48808 записей:
OCR тексты:          ['3начит, там, где растут эти дёревья, вечпые лнхорадки…  –Вовсе нет', '–А когда это я ныл по пустякам', 'К моему и квашему несчастью, я не в силах освободить вас из злого плена'] ...
Эталонные тексты:    ['Значит, там, где растут эти деревья, вечные лихорадки…  –Вовсе нет', '–А когда это я ныл по пустякам', 'К моему и квашему несчастью, я не в силах освободить вас из злого плена'] ...
Исправленные тексты: ['3начит, там, где растут эти дёревья, вечные лнхорадки…  –Вовсе нет', '–А когда это я был по пустякам', 'К моему и вашему несчастью, я не в силах освободить вас из злого плена'] ...


Метрики для текста сгенерированных OCR-ошибок.

In [ ]:
wer_ocr, cer_ocr = metrics_wer_cer(correct_texts, ocr_texts)
print(f"WER: {wer_ocr}")
print(f"CER: {cer_ocr}")

WER: 0.333
CER: 0.051


Метрики для текста с автокоррекцией с дообученной моделью.

In [ ]:
wer_corr, cer_corr = metrics_wer_cer(correct_texts, corrected_texts)
print(f"WER: {wer_corr}")
print(f"CER: {cer_corr}")

WER: 0.304
CER: 0.049


**ВЫВОД:**

Загружены несколько текстов и был сгенерирован датасет с синтетическими OCR-ошибками размером **104549** предложений. Модель **JamSpell** была дообучена на этом датасете, выведены результаты для сравнения и вычислены метрики WER и CER.

**До дообучения (текст с OCR-ошибками):**
WER: 0.326 и CER: 0.051

**После дообучения (тестовый текст):**
WER: 0.295 и CER: 0.048

Дообучение модели **JamSpell** дало плохие результаты, несмотря на то, что в предыдущей работе по исследованию автокорректоров она показала лучшие результаты. Даже визуально очевидно, что ошибки исправлены плохо.

Возможные причины:
- датасет не достаточного объема для лучшего обучения,
- ошибки сгенерированы не корректно,
- тип модели не позволяет более эффективно обучить модель.
Необходимо попробовать дообучить другую модель на основе нейросети с переносом расчетов на ГПУ.

Датасет увеличен до **244033** предложений, однако, улучшения метрик это не дало.

**До дообучения (текст с OCR-ошибками):**
WER: 0.333 и CER: 0.051

**После дообучения (тестовый текст):**
WER: 0.304 и CER: 0.049

Предполагаю, что надо пропробовать заменить модель на более эффективную, например, модель RuM2M100-1.2B. Однако, сложность работы с этой моделью будет в том, что она требует переноса на ГПУ и большой ресурс видеопамяти.